In [ ]:
d=2
k=2
sigma = 0.1


def eq(x,y):
    return abs(x-y)<0.00001




def com(points, mask, values):
    total_sum = 0
    total_elements = 0
    for i,m in enumerate(mask):
        if (m):
            total_sum+=histogram[i]*values[i]
            total_elements+= histogram[i]
    return total_sum/total_elements
    

def kmeans_loss(histogram, values, mu):
    """
    @param: mu an array of cluster centers
    @param: histogram[locID]= amount of points in location whose index is locID
    """
    loss = 0
    for i, count in enumerate(histogram):
        value = values[i]
        min_dist = min(abs(value-mu[0]), abs(value-mu[1]))
        loss += min_dist**2 * count
    return loss

def counterexample_calc_2means(histogram):
    """
    @param: histogram[locID]= amount of points in location whose index is locID
    """
    muRight = [com(histogram,CLUSTER_MASK_eps, VALUES),1]
    muLeft = [-eps,com(histogram,CLUSTER_MASK_1, VALUES)]
    return muLeft if kmeans_loss(histogram, VALUES, muLeft)<kmeans_loss(histogram,VALUES, muRight) else muRight


class FTL:
    def __init__(self, histogram):
        self.mu = self.choose_best(histogram)
        if (eq(max(self.mu),1)):
            self.alignment = ALIGNED_RIGHT
        else:
            assert(eq(min(self.mu),-eps))
            self.alignment = ALIGNED_LEFT
    
    def choose_best(self, histogram):
        if (sum(histogram)<2):
            return [-eps,0]
        else:
            return counterexample_calc_2means(histogram)
    
    def loss(self, x_t):
        tmp_hist = [0,0,0]
        tmp_hist[x_t]=1
        l_t = kmeans_loss(tmp_hist, VALUES, self.mu)
        return l_t




def FTL_adversary(ftl_obj, t, T):
    """
    @param: ftl_obj.histogram[locID]= amount of points in location whose index is locID
    @return: locID of next point in stream
    """
    if (t>T/2):
        return oneID
    #print(ftl_obj.mu, ftl_obj.alignment == ALIGNED_RIGHT)
    if (t==0):
        return epsID
    elif (t==1):
        return zeroID
         
    if (ftl_obj.alignment == ALIGNED_RIGHT):
        return epsID if min(ftl_obj.mu)>-eps/2 else zeroID
    else:
        assert(ftl_obj.alignment == ALIGNED_LEFT)
        return oneID


    
def analyze(histogram, online_loss, shouldPrint=True):
    prophet_loss = kmeans_loss(histogram, VALUES, counterexample_calc_2means(histogram))    
    regret = online_loss - prophet_loss
    
    if (shouldPrint):
        print('online loss: ', online_loss)
        print('prophet loss: ', prophet_loss)
        print('regret=',online_loss - prophet_loss)
    return (online_loss, prophet_loss, regret)